In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [2]:
import torch
import src.load as load
import numpy as np

import app.utils as utils
import pandas as pd
import src.table as table


In [3]:
folder_path = "../app/models"

In [4]:
models = utils.load_models(folder_path)


/Users/zakir/anaconda3/envs/cherry/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [5]:
train_dataset, test_dataset, mapping_dict, reverse_mappings, train_meta, test_meta = load.separate_year('../data/processed/')

In [6]:
transplant_weeks = test_meta['WeekTransplanted']

In [7]:
pred_path = "../app/preds"

In [12]:
week_list = os.listdir(pred_path)

In [17]:
num_preds = len(week_list)
num_plantings = len(test_dataset)

In [14]:
num_weeks = 40

In [16]:
transplant_weeks = test_meta['WeekTransplanted']

In [26]:
idxs = table.create_indices_dict(test_meta)
bsuf_idx = idxs['Class']['BSUF']

In [78]:
agg_kilo_preds = np.zeros((num_plantings, num_preds,num_weeks))
agg_sched_preds = np.zeros((num_plantings, num_preds, 5))

In [81]:
for week in week_list:
    week_path = os.path.join(pred_path, week)
    week_idx = int(week.split('_')[-1]) - 1
    for file in os.listdir(week_path):
        if 'kilo_preds' in file:
            preds = pd.read_csv(os.path.join(week_path, file),index_col=0).values
            agg_kilo_preds[:,week_idx,:] = preds
        elif 'sched_preds' in file:
            preds = pd.read_csv(os.path.join(week_path, file),index_col=0).values
            agg_sched_preds[:,week_idx,:] = preds
        else:
            print('eeker casillas')

In [83]:
agg_kilo_preds

array([[[ 6.24236298e+01,  2.93248348e+01, -3.07660255e+01, ...,
          1.95476489e+03,  1.75148486e+03,  1.79947620e+03],
        [ 6.24236298e+01,  2.93248348e+01, -3.07660255e+01, ...,
          1.95476489e+03,  1.75148486e+03,  1.79947620e+03],
        [ 6.24236298e+01,  2.93248348e+01, -3.07660255e+01, ...,
          1.95476489e+03,  1.75148486e+03,  1.79947620e+03],
        ...,
        [ 2.20625706e+01, -1.54256149e+02, -6.21294098e+01, ...,
          2.20714893e+03,  2.11360767e+03,  2.29231030e+03],
        [ 2.20625706e+01, -1.54256149e+02, -6.21294098e+01, ...,
          2.20714893e+03,  2.11360767e+03,  2.29231030e+03],
        [ 2.20625706e+01, -1.54256149e+02, -6.21294098e+01, ...,
          2.20714893e+03,  2.11360767e+03,  2.29231030e+03]],

       [[ 9.20294285e+00, -1.34544449e+01,  1.88214636e+00, ...,
          6.93985779e+02,  7.16510925e+02,  6.88871582e+02],
        [ 9.20294285e+00, -1.34544449e+01,  1.88214636e+00, ...,
          6.93985779e+02,  7.16510925e

In [84]:
actual_kilos = np.round((torch.exp(test_dataset.yield_dist.view(-1, 40)) - 1).numpy() * test_dataset.features[:,0].view(-1,1).numpy())
actual_sched = test_dataset.Y_schedule.numpy()

In [74]:
actual_kilos.shape

(176, 40)